In [ ]:
import os
import openai
from azure.identity import DefaultAzureCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import QueryType
from langchain.agents import Tool, AgentExecutor
from langchain.agents.react.base import ReActDocstoreAgent
from langchain.llms.openai import AzureOpenAI
from langchain.prompts import PromptTemplate, BasePromptTemplate
from typing import List

# Replace these with your own values, either in environment variables or directly here
AZURE_STORAGE_ACCOUNT = os.environ.get("AZURE_STORAGE_ACCOUNT") or "mystorageaccount"
AZURE_STORAGE_CONTAINER = os.environ.get("AZURE_STORAGE_CONTAINER") or "content"
AZURE_SEARCH_SERVICE = os.environ.get("AZURE_SEARCH_SERVICE") or "gptkb"
AZURE_SEARCH_INDEX = os.environ.get("AZURE_SEARCH_INDEX") or "gptkbindex"
AZURE_OPENAI_SERVICE = os.environ.get("AZURE_OPENAI_SERVICE") or "myopenai"
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT_DEPLOYMENT") or "davinci"

KB_FIELDS_CONTENT = os.environ.get("KB_FIELDS_CONTENT") or "content"
KB_FIELDS_CATEGORY = os.environ.get("KB_FIELDS_CATEGORY") or "category"
KB_FIELDS_SOURCEPAGE = os.environ.get("KB_FIELDS_SOURCEPAGE") or "sourcepage"

# Use the current user identity to authenticate with Azure OpenAI, Cognitive Search and Blob Storage (no secrets needed, 
# just use 'az login' locally, and managed identity when deployed on Azure). If you need to use keys, use separate AzureKeyCredential instances with the 
# keys for each service
azure_credential = DefaultAzureCredential()

# Used by the OpenAI SDK
openai.api_type = "azure"
openai.api_base = f"https://{AZURE_OPENAI_SERVICE}.openai.azure.com"
openai.api_version = "2022-12-01"

# Comment these two lines out if using keys, set your API key in the OPENAI_API_KEY environment variable instead
openai.api_type = "azure_ad"
openai.api_key = azure_credential.get_token("https://cognitiveservices.azure.com/.default").token

# Set up clients for Cognitive Search and Storage
search_client = SearchClient(
    endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    index_name=AZURE_SEARCH_INDEX,
    credential=azure_credential)

In [ ]:
# Modified version of langchain's ReAct prompt that includes instructions and examples for how to cite information sources

EXAMPLES = [
    """Pytanie: Jaki jest zakres wysokości dla obszaru, na który rozciąga się wschodni sektor orogenezy Kolorado?
Myśl: Muszę przeszukać orogenezę Kolorado, znaleźć obszar, na który rozciąga się wschodni sektor orogenezy Kolorado, 
a następnie znaleźć zakres wysokości tego obszaru.
Akcja: Szukaj[orogeneza Kolorado]
Obserwacja: <info1.pdf> Orogeneza w Kolorado była epizodem budowania gór (orogenezy) w Kolorado i okolicach.
Myśl: Nie wspomina o sektorze wschodnim. Więc muszę poszukać wschodniego sektora.
Akcja: Lookup [sektor wschodni]
Obserwacja: <info2.txt> (Rezultat 1 / 1) Wschodni sektor rozciąga się na High Plains i nazywany jest orogenezą Równin Centralnych.
Myśl: Wschodni sektor orogenezy Kolorado rozciąga się na High Plains. Więc muszę przeszukać High Plains i znaleźć zakres wzniesień.
Akcja: Szukaj[High Plains]
Obserwacja: <some_file.pdf> High Plains odnosi się do jednego z dwóch odrębnych regionów lądowych
Akcja: zamiast tego muszę przeszukać High Plains (Stany Zjednoczone).
Action: Szukaj[High Plains (Stany Zjednoczone)]
Obserwacja: <plik.pdf> High Plains są podregionem Great Plains. <another-ref.docx> 
Ze wschodu na zachód High Plains wznoszą się od około 1800 do 7000 stóp (550 do 2130 m).
Myśl: High Plains wznoszą się od około 1800 do 7000 stóp, więc odpowiedź brzmi od 1800 do 7000 stóp.
Akcja: Finish[1800 do 7000 stóp <plik.pdf>]""",
    """Pytanie: Allie Goertz, muzyk i satyryk, napisała piosenkę o Milhouse'ie, bohaterze „The Simpsons”, którego imieniem Matt Groening nazwał na cześć kogo?
Myśl: Pytanie upraszcza się do tego, że Milhouse, postać z „Simpsonów”, nosi imię kogo. Muszę tylko przeszukać Milhouse i znaleźć jego imię.
Akcja: Szukaj[Milhouse]
Obserwacja: <info7.pdf> Milhouse Mussolini Van Houten to powracająca postać w animowanym serialu telewizyjnym Fox The Simpsons, 
nagłośnionym przez Pamelę Hayden i stworzonym przez Matta Groeninga.
Myśl: Akapit nie mówi, po kim Milhouse jest nazwany, może uda mi się wyszukać „nazwany po”.
Akcja: Przeszukaj[nazwany po]
Obserwacja: <historyref2.txt> (Wynik 1 / 1) Milhouse został nazwany na cześć prezydenta USA Richarda Nixona, którego drugie imię brzmiało Milhous.
Myśl: Milhouse został nazwany na cześć prezydenta USA Richarda Nixona, więc odpowiedź brzmi: Richard Nixon.
Akcja: Finish[Richard Nixon <historyref2.txt>]""",
    """Pytanie: Który dokument dotyczy fińskich grup rockowych, Adama Claytona Powella lub The Saimaa Gesture?
Myśl: Muszę poszukać Adama Claytona Powella i The Saimaa Gesture i znaleźć dokument o fińskich grupach rockowych.
Akcja: Szukaj[Adam Clayton Powell]
Obserwacja: <info9.pdf> Nie można znaleźć [Adam Clayton Powell]. Podobne: ['Adam Clayton Powell III', 'Seventh Avenue (Manhattan)', 
'Adam Clayton Powell Jr. State Office Building', 'Isabel Washington Powell', 
'Adam Powell', Adam Clayton Powell (film)', 'Giancarlo Esposito'].
Myśl: Aby znaleźć dokument, mogę przeszukać Adama Claytona Powella (film).
Akcja: Szukaj[Adam Clayton Powell (film)]
Obserwacja: <data123.txt> Adam Clayton Powell to amerykański film dokumentalny z 1989 roku, 
wyreżyserowany przez Richarda Kilberga. Film opowiada o wzlotach i upadkach wpływowego afroamerykańskiego 
polityka Adama Claytona Powella Jr. Został później wyemitowany jako część serii PBS The American Experience .
Myśl: Adam Clayton Powell (film) to dokument o afroamerykańskim polityku, a nie fińskich grupach rockowych. 
Tak więc film dokumentalny o fińskich grupach rockowych musi być zamiast tego The Saimaa Gesture.
Akcja: Finish[The Saimaa Gesture <data123.txt>]""",
    """Pytanie: Jaki zawód łączy Nicholas Ray i Elia Kazan?
Myśl: Muszę odszukać Nicholasa Raya i Elię Kazana, znaleźć ich zawody, a potem znaleźć zawód, który ich łączy.
Akcja: Szukaj[Nicholas Ray]
Obserwacja: <files-987.png> Nicholas Ray (ur. Raymond Nicholas Kienzle Jr., 7 sierpnia 1911 - 16 czerwca 1979) był amerykańskim reżyserem filmowym, 
scenarzystą i aktorem najbardziej znanym z filmu Buntownik bez powodu z 1955 roku.
Myśl: Zawody Nicholasa Raya to reżyser, scenarzysta i aktor. W następnej kolejności muszę przeszukać Elię Kazana i znaleźć jego profesje.
Akcja: Szukaj[Elia Kazan]
Obserwacja: <files-654.txt> Elia Kazan był amerykańskim reżyserem filmowym i teatralnym, producentem, scenarzystą i aktorem.
Myśl: Zawody Elii Kazana to reżyser, producent, scenarzysta i aktor. Tak więc wspólny zawód Nicholasa Raya i Elii Kazana to reżyser, scenarzysta i aktor.
Akcja: Finish[reżyser, scenarzysta, aktor <files-987.png><files-654.txt>]""",
    """Pytanie: Który magazyn powstał jako pierwszy Arthur's Magazine czy First for Women?
Myśl: Muszę przeszukać Arthur's Magazine i First for Women i znaleźć, który został uruchomiony jako pierwszy.
Akcja: Szukaj[Arthur's Magazine]
Obserwacja: <magazines-1850.pdf> Arthur's Magazine (1844-1846) był amerykańskim czasopismem literackim wydawanym w Filadelfii w XIX wieku.
Myśl: Arthur's Magazine został uruchomiony w 1844 roku. W następnej kolejności muszę wyszukać First for Women.
Akcja: Szukaj[First for Women]
Obserwacja: <magazines-1900.pdf> First for Women to kobiecy magazyn wydawany przez Bauer Media Group w USA.[1] Czasopismo powstało w 1989 roku.
Myśl: First for Women powstał w 1989 roku. 1844 (Arthur's Magazine) < 1989 (First for Women), więc Arthur's Magazine powstał jako pierwszy.
Akcja: Finish[Arthur's Magazine <magazines-1850.pdf><magazines-1900.pdf>]""",

    """Pytanie: Czy Pavel Urysohn i Leonid Levin byli znani z tego samego rodzaju pracy?
Myśl: Muszę przeszukać Pavel Urysohn i Leonid Levin, znaleźć ich rodzaj pracy, a potem sprawdzić, czy to to samo.
Akcja: Szukaj[Pavel Urysohn]
Observation: <info4444.pdf> Pavel Samuilovich Urysohn (3 lutego 1898 - 17 sierpnia 1924) był radzieckim matematykiem, 
najbardziej znanym ze swojego wkładu w teorię wymiarów.
Thought: Pavel Urysohn jest matematykiem. Muszę następnie wyszukać Leonida Levina i znaleźć jego rodzaj pracy.
Akcja: Szukaj[Leonid Levin]
Obserwacja: <datapoints_aaa.txt> Leonid Anatolievich Levin jest radziecko-amerykańskim matematykiem i informatykiem.
Thought: Leonid Levin jest matematykiem i informatykiem. Tak więc Pavel Urysohn i Leonid Levin mają ten sam rodzaj pracy.
Akcja: Finish[Tak <info4444.pdf><datapoints_aaa.txt>]""",
]
SUFFIX = """\Pytanie: {input}
{agent_scratchpad}"""
PREFIX = "Odpowiadaj na pytania, jak pokazano w poniższych przykładach, dzieląc pytanie na poszczególne czynności szukania lub przeszukiwania, aby znaleźć fakty, dopóki nie będziesz w stanie odpowiedzieć na pytanie." \
"Obserwacje są poprzedzone nazwą źródła w nawiasach ostrych, nazwy źródeł MUSZĄ być dołączone do działań w odpowiedziach." \
"Na wszystkie pytania należy odpowiedzieć na podstawie wyników działań wyszukiwania lub przeszukiwania, w odpowiedzi można wykorzystać tylko wynikające z nich fakty."
"Answer questions as truthfully as possible, and ONLY answer the questions using the information from observations, do not speculate or your own knowledge."

prompt = PromptTemplate.from_examples(
    EXAMPLES, SUFFIX, ["input", "agent_scratchpad"], PREFIX
)

In [ ]:
# Exclude category, to simulate scenarios where there's a set of docs you can't see
exclude_category = None

def search(terms: str):
    print ("\nPrzeszukiwanie: " + terms)
    # Optionally enable captions for summaries by adding optional arugment query_caption="extractive|highlight-false"
    # and adjust the string formatting below to include the captions from the @search.captions field
    filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None
    r = search_client.search(terms, 
                             filter=filter,
                             top = 3,
                             query_type=QueryType.SEMANTIC, 
                             query_language="pl-pl", 
                             query_speller="lexicon", 
                             semantic_configuration_name="default")
    return "\n".join([f"<{doc[KB_FIELDS_SOURCEPAGE]}> " + (doc[KB_FIELDS_CONTENT][:500]).replace("\n", "").replace("\r", "") for doc in r])

def lookup(terms: str):
    print ("\nlooking up: " + terms)
    filter = "category ne '{}'".format(exclude_category.replace("'", "''")) if exclude_category else None
    r = search_client.search(terms, 
                             filter=filter,
                             top = 1,
                             include_total_count=True,
                             query_type=QueryType.SEMANTIC, 
                             query_language="pl-pl", 
                             query_speller="lexicon", 
                             semantic_configuration_name="default",
                             query_answer="extractive|count-1",
                             query_caption="extractive|highlight-false")
    answers = r.get_answers()
    if len(answers) > 0:
        return answers[0].text
    if r.get_count() > 0:
        return "\n".join(c.text for c in next(r)["@search.captions"])
    return None

llm = AzureOpenAI(deployment_name=AZURE_OPENAI_GPT_DEPLOYMENT, temperature=0.3, openai_api_key=openai.api_key)
tools = [
    Tool(name="Szukaj", func=search, description="useful for when you need to ask with search"),
    Tool(name="Przeszukaj", func=lookup, description="useful for when you need to ask with lookup"  )
]

class ReAct(ReActDocstoreAgent):
    @classmethod
    def create_prompt(cls, tools: List[Tool]) -> BasePromptTemplate:
        return prompt

agent = ReAct.from_llm_and_tools(llm, tools)
chain = AgentExecutor.from_agent_and_tools(agent, tools, verbose=True)

In [ ]:
# Exclude category, to simulate scenarios where there's a set of docs you can't see
exclude_category = None

chain.run("Ile sprzedano kredytów mikroprzedsiębiorcom w grudniu 2022 w porownaniu z grudniem 2021?")